In [1]:
# Assignment 1_4 POS Tagging - HMM

# -*- coding: utf-8 -*-
import os
import re
import math
import random
import numpy as np

In [2]:
"""Get all files name under path

Args:
    path: file path.
    suffix: type of files
    shuffle: a boolean value. TRUE to shuffle list and False to order list.

Returns:
    filesName[:train]: a list of all files end with ".txt" for training set. For example:
    
    ["dir/a.txt", "dir/b.txt"]
"""
def getFilesName( path, suffix = "", shuffle = False ):
    print( "Retrieving files name from folder %s..." % ( path ) )
    filesName = []
    files = os.listdir( path )
    for file in files:
        if os.path.splitext( file )[1] == suffix:
            name = '/'.join( [path, file] )
            filesName.append( name )
    if shuffle:
        random.shuffle( filesName )
    else:
        filesName.sort()
    return filesName

In [3]:
# Problem 4.1
"""Word Tag Count

Obtain frequency counts from the collection of all the training files,
including word-tag counts, tag unigram counts, and tag bigram counts.

Args:
    content: content need to be counted.
    dwt: a dictionary on counting word-tag. Its structure is:

        {tag: {word: count}}.

    dt : a dictionary on counting tag unigram. Its structure is:

        {tag: count}.

    dtt: a dictionary on counting tag bigram. Its structure is:

        {cur_tag|prev_tag: count}.

Returns:
    None.
"""
def countWordTag( content, dwt, dt, dtt ):
    wt = re.split( "\s", content )
    wt = list( filter( None, wt ) )
    if len( wt ) == 0:
        return dwt, dt, dtt
    wt = ["<SOS>/<SOS>"] + wt + ["<EOS>/<EOS>"]
    n = len( wt )
    for i in range( n ):
        # word-tag count
        if wt[i] not in dwt:
            dwt[wt[i]] = 0
        dwt[wt[i]] += 1
        
        # tag unigram counts
        ti  = wt[i].split( '/' )
        tag = ti[-1]
        if tag not in dt:
            dt[tag] = 0
        dt[tag] += 1
        
        # tag bigram counts
        if i:
            pti  = wt[i - 1].split( '/' )
            ptag = pti[-1]
            tt = '|'.join( [tag, ptag] )
            if tt not in dtt:
                dtt[tt] = 0
            dtt[tt] += 1

"""Main function of problem 4.1

Counting word-tag, tag unigram, tag bigram and write in corresponding files.

dwt: a dictionary on counting word-tag. Saved in file "word-tag". Its structure is:

    {word/tag: count}.

dt : a dictionary on counting tag unigram. Saved in file "tag-unigram". Its structure is:

    {tag: count}.

dtt: a dictionary on counting tag bigram. Saved in file "tag-bigram". Its structure is:

    {cur_tag|prev_tag: count}

Args:
    trainDataPath: train data path.
    encoding: train data files' encoding.
    savePath: path to save count files.

Returns:
    None.
"""
def problem4_1( trainDataPath = "./Brown", encoding = "UTF-8", savePath = "./4_1" ):
    filesName = getFilesName( trainDataPath )
    dwt = {}
    dt  = {}
    dtt = {}
    for fileName in filesName:
        print( "Counting on %s" % fileName )
        with open( fileName, 'r', encoding = encoding ) as f:
            line = f.readline()
            while line:
                countWordTag( line, dwt, dt, dtt )
                line = f.readline()
    dwt = sorted( dwt.items(), key = lambda x: x[1], reverse = True )
    dt  = sorted( dt .items(), key = lambda x: x[1], reverse = True )
    dtt = sorted( dtt.items(), key = lambda x: x[1], reverse = True )

    with open( savePath + "/" + "word-tag", 'w' ) as f:
        for kv in dwt:
            f.write( kv[0] + ' ' + str( kv[1] ) + '\n' )
    with open( savePath + "/" + "tag-unigram", 'w' ) as f:
        for kv in dt:
            f.write( kv[0] + ' ' + str( kv[1] ) + '\n' )
    with open( savePath + "/" + "tag-bigram", 'w' ) as f:
        for kv in dtt:
            f.write( kv[0] + ' ' + str( kv[1] ) + '\n' )

In [4]:
# Problem 4.2
"""Emission probability
dt : a dictionary on counting tag unigram. Its structure is:

    {tag: count}.

dtt: a dictionary on counting tag bigram. Its structure is:

    {cur_tag|prev_tag: count}.

Args:
    filePath: path containing counting files.
    encoding: counting files' encoding.
    lambd: a parameter for lambda smoothing.

Returns:
    dptt: a dictionary of probability of tag bigram. Its structure is:
    
    {cur_tag|prev_tag: probability}.
"""
def transitProb( filePath = "./4_1", encoding = "UTF-8", lambd = 0.1 ):
    dt   = {}
    dtt  = {}
    dptt = {}
    with open( filePath + "/" + "tag-unigram", 'r', encoding = encoding ) as f:
        line = f.readline()
        while line:
            line = line.split( ' ' )
            dt[line[0]] = int( line[1] )
            line = f.readline()
    with open( filePath + "/" + "tag-bigram", 'r', encoding = encoding ) as f:
        line = f.readline()
        while line:
            line = line.split( ' ' )
            dtt[line[0]] = int( line[1] )
            line = f.readline()
    
    # calculate emission probability
    for ( k, v ) in dtt.items():
        ptag = k.split( '|' )[-1]
        dptt[k] = ( v + lambd ) / ( dt[ptag] + lambd * len( dt ) )
    return dptt

In [5]:
# Problem 4.3
"""Emission probability

dt : a dictionary on counting tag unigram. Its structure is:

    {tag: count}.

dwt: a dictionary on counting word-tag. Its structure is:

    {word|tag: count}.

Args:
    filePath: path containing counting files.
    encoding: counting files' encoding.
    lambd: a parameter for lambda smoothing.

Returns:
    dpwt: a dictionary of probability of word-tag. For example:
    
        {word|tag: probability}.
"""
def emitProb( filePath = "./4_1", encoding = "UTF-8", lambd = 0.1 ):
    dt   = {}
    dwt  = {}
    dpwt = {}
    with open( filePath + "/" + "tag-unigram", 'r', encoding = encoding ) as f:
        line = f.readline()
        while line:
            line = line.split( ' ' )
            dt[line[0]] = int( line[-1] )
            line = f.readline()
    with open( filePath + "/" + "word-tag", 'r', encoding = encoding ) as f:
        line = f.readline()
        while line:
            line = line.split( ' ' )
            dwt[line[0]] = int( line[-1] )
            line = f.readline()

    # calculate emission probability
    for ( k, v ) in dwt.items():
        tag = k.split( '/' )[-1]
        dpwt[k] = ( v + lambd ) / ( dt[tag] + lambd * len( dt ) )
    return dpwt

In [6]:
# Problem 4.4
"""Read transition probs matrix and emission probs matrix.

Args:
    filePath: path containing word-tag counting and tag bigram counting.
    encoding: train data files' encoding.

Returns:
    dwt: a dictionary on counting word-tag. Its structure is:

        {tag: {word: count}}.

    dtt: a dictionary on counting tag bigram. Its structure is:

        {prev_tag: {cur_tag: count}}.
"""
def probs( filePath = "./4_1", encoding = "UTF-8" ):
    dwt = {}
    dtt = {}
    with open( filePath + "/" + "word-tag", 'r', encoding = encoding ) as f:
        line = f.readline()
        while line:
            line = line.split( ' ' )
            v = int( line[-1] )
            line = line[0].split( '/' )
            w = line[0]
            t = line[-1]
            if t not in dwt:
                dwt[t] = {}
            dwt[t][w] = v
            line = f.readline()
    with open( filePath + "/" + "tag-bigram", 'r', encoding = encoding ) as f:
        line = f.readline()
        while line:
            line = line.split( ' ' )
            v = int( line[-1] )
            line = line[0].split( '|' )
            t1 = line[-1]
            t2 = line[0]
            if t1 not in dtt:
                dtt[t1] = {}
            dtt[t1][t2] = v
            line = f.readline()
    return dwt, dtt

"""Generate sample sentence

Args:
    dwt: a dictionary on counting word-tag. Its structure is:

        {tag: {word: count}}.

    dtt: a dictionary on counting tag bigram. Its structure is:

        {prev_tag: {cur_tag: count}}.
    
    dpwt: a dictionary contains probabilities of emission. Its structure is:
    
        {word|tag: probability}.
    
    dptt: a dictionary contains probabilities of transition. Its structure is:

        {cur_tag|prev_tag: probability}.

Returns:
    s: generated sentence.
    tag: generated tag sequence.
"""
def generate( mseed, dwt, dtt, dpwt, dptt ):
    random.seed( mseed )
    tag = "<SOS>"
    p = 0
    s = ""
    while tag != "<EOS>":
        ptag = tag
        # generate new tag
        tag  = str( random.choices( list( dtt[tag].keys() ),
                                    list( dtt[tag].values() ), k = 1 )[0] )
        if tag == "<EOS>":
            break;
        # generate new word
        word = str( random.choices( list( dwt[tag].keys() ),
                                    list( dwt[tag].values() ), k = 1 )[0] )
        s += " " + word
        tt = '|'.join( [tag, ptag] )
        wt = '/'.join( [word, tag] )
        p += math.log( dptt[tt] ) + math.log( dpwt[wt] )
    s = s.strip()
    p = math.e ** p
    return s, p

"""Main function of problem 4.4

Generate sample sentences based on HMM transition and emission probability.

Args:
    filePath: train data path.
    encoding: train data files' encoding.

Returns:
    None.
"""
def problem4_4( filePath = "./4_1", encoding = "UTF-8" ):
    dwt, dtt = probs( filePath, encoding )
    dptt = transitProb( filePath, encoding )
    dpwt = emitProb( filePath, encoding)
    for i in range( 5 ):
        print( generate( i, dwt, dtt, dpwt, dptt ) )

In [7]:
# Problem 4.5
"""Calculate current status probability

Calculate new status d[i, s]:

dp[i, s] = dp[i - 1, s'] * t( t1|t2 ) * e( s|t1 ).

Args:
    dt : a dictionary on counting tag unigram. Its structure is:

        {tag: count}.

    dpwt: a dictionary contains probabilities of emission. Its structure is:
    
        {word|tag: probability}.
    
    dptt: a dictionary contains probabilities of transition. Its structure is:

        {cur_tag|prev_tag: probability}.
    
    p: previous status.
    t2: previous hidden tag.
    t1: current hidden tag.
    s: current word in sentence.
    lambd: used for add-lambda smoothing.

Returns:
    p: new status.
"""
def cal( dt, dptt, dpwt, p, t2, t1, s, lambd ):
    try:
        p *= dptt['|'.join( [t1, t2] )]
    except:
        try:
            p *= lambd / ( dt[t2] + lambd * len( dt ) )
        except:
            p *= 1 / len( dt )
    try:
        p *= dpwt['/'.join( [s , t1] )]
    except:
        try:
            p *= lambd / ( dt[t1] + lambd * len( dt ) )
        except:
            p *= 1 / len( dt )
    return p

"""Viterbi algorithm

Args:
    s: sentece need to be parsed.
    dt : a dictionary on counting tag unigram. Its structure is:

        {tag: count}.

    dpwt: a dictionary contains probabilities of emission. Its structure is:
    
        {word|tag: probability}.
    
    dptt: a dictionary contains probabilities of transition. Its structure is:

        {cur_tag|prev_tag: probability}.
    
    lambd: lambda used for add-lambda smoothing.

Returns:
    tags: parsed tags from given sentence s.
"""
def viterbi( s, dt, dptt, dpwt, lambd = 0.1 ):
    # initialize
    t = list( dt.keys() )
    n = len( t )
    length = len( s )
    dp = []
    bt = []
    
    # calcuate the first word
    dp.append( [0] * n )
    bt.append( [0] * n )
    for i in range( n ):
        p = cal( dt, dptt, dpwt, 1, "<SOS>", t[i], s[0], lambd )
        if p > dp[0][i]:
            dp[0][i] = p
            bt[0][i] = 0

    # calculate word between the second and the last
    for i in range( 1, length - 1 ):
        for j in range( n ):
            dp.append( [0] * n )
            bt.append( [0] * n )
            for k in range( n ):
                p = cal( dt, dptt, dpwt, dp[i - 1][k], t[k], t[j], s[i], lambd )
                if p > dp[i][j]:
                    dp[i][j] = p
                    bt[i][j] = k
    
    # calculate <EOS>
    dp.append( [0] * n )
    bt.append( [0] * n )
    for i in range( n ):
        p = cal( dt, dptt, dpwt, dp[length-2][i], t[i], "<EOS>", s[length-1], lambd )
        if p > dp[length-1][0]:
            dp[length-1][0] = p
            bt[length-1][0] = i
    
    # back track
    nmax = dp[length-1][0]
    tag = bt[length-1][0]
    tags = ["<EOS>"]
    for i in range( length - 2, -1, -1 ):
        tags.append( t[tag] )
        tag = bt[i][tag]
    tags.reverse()
    
    return tags

"""Main function of problem 4.5

For given sentences, the functino use viterbi algorithm to parse its POS tags.

Args:
    filePath: the path contains HMM files.
    encoding: the encoding of to be processed files.
    savePath: the path that tags will save.
    testFilePath: the path contains test files.

Returns:
    None.
"""
def problem4_5( filePath = "./4_1", encoding = "UTF-8", savePath = "./4_5",
                   testFilePath = "." ):
    # load all HMM parameters
    dt = {}
    with open( filePath + "/" + "tag-unigram", 'r', encoding = encoding ) as f:
        line = f.readline()
        while line:
            line = line.split( ' ' )
            dt[line[0]] = int( line[-1] )
            line = f.readline()
    with open( filePath + "/" + "tag-unigram", 'r', encoding = encoding ) as f:
        line = f.readline()
        while line:
            line = line.split( ' ' )
            dt[line[0]] = int( line[-1] )
            line = f.readline()
    dptt = transitProb( filePath, encoding )
    dpwt = emitProb( filePath, encoding )

    # calculate POS tag
    content = ""
    s = []
    f1 = open( testFilePath + "/" + "Test_File.txt", 'r', encoding = encoding )
    f2 = open( savePath + "/" + "test_result", 'w', encoding = encoding )
    line = f1.readline()
    while line:
        if line[:3] == "< s":
            f2.write( line[0] + line[2:13] + line[14:] )
        else:
            s.append( line.strip() )
        if line.strip() == "<EOS>":
            print( "Ori: " + str( s ) )
            tags = viterbi( s, dt, dptt, dpwt )
            print( "Tag: " + str( tags ) )
            for i in range( len( tags ) - 1 ):
                f2.write( s[i] + ", " + tags[i] + "\n" )
            f2.write( "<EOS>\n" )
            s = []
        line = f1.readline()
    f1.close()
    f2.close()

In [8]:
# problem 4.1
print( "Running on problem 4.1" )
problem4_1( trainDataPath = "./Brown", encoding = "UTF-8", savePath = "./4_1" )

Running on problem 4.1
Retrieving files name from folder ./Brown...
Counting on ./Brown/ca01
Counting on ./Brown/ca02
Counting on ./Brown/ca03
Counting on ./Brown/ca04
Counting on ./Brown/ca05
Counting on ./Brown/ca06
Counting on ./Brown/ca07
Counting on ./Brown/ca08
Counting on ./Brown/ca09
Counting on ./Brown/ca10
Counting on ./Brown/ca11
Counting on ./Brown/ca12
Counting on ./Brown/ca13
Counting on ./Brown/ca14
Counting on ./Brown/ca15
Counting on ./Brown/ca16
Counting on ./Brown/ca17
Counting on ./Brown/ca18
Counting on ./Brown/ca19
Counting on ./Brown/ca20
Counting on ./Brown/ca21
Counting on ./Brown/ca22
Counting on ./Brown/ca23
Counting on ./Brown/ca24
Counting on ./Brown/ca25
Counting on ./Brown/ca26
Counting on ./Brown/ca27
Counting on ./Brown/ca28
Counting on ./Brown/ca29
Counting on ./Brown/ca30
Counting on ./Brown/ca31
Counting on ./Brown/ca32
Counting on ./Brown/ca33
Counting on ./Brown/ca34
Counting on ./Brown/ca35
Counting on ./Brown/ca36
Counting on ./Brown/ca37
Countin

Counting on ./Brown/cj53
Counting on ./Brown/cj54
Counting on ./Brown/cj55
Counting on ./Brown/cj56
Counting on ./Brown/cj57
Counting on ./Brown/cj58
Counting on ./Brown/cj59
Counting on ./Brown/cj60
Counting on ./Brown/cj61
Counting on ./Brown/cj62
Counting on ./Brown/cj63
Counting on ./Brown/cj64
Counting on ./Brown/cj65
Counting on ./Brown/cj66
Counting on ./Brown/cj67
Counting on ./Brown/cj68
Counting on ./Brown/cj69
Counting on ./Brown/cj70
Counting on ./Brown/cj71
Counting on ./Brown/cj72
Counting on ./Brown/cj73
Counting on ./Brown/cj74
Counting on ./Brown/cj75
Counting on ./Brown/cj76
Counting on ./Brown/cj77
Counting on ./Brown/cj78
Counting on ./Brown/cj79
Counting on ./Brown/cj80
Counting on ./Brown/ck01
Counting on ./Brown/ck02
Counting on ./Brown/ck03
Counting on ./Brown/ck04
Counting on ./Brown/ck05
Counting on ./Brown/ck06
Counting on ./Brown/ck07
Counting on ./Brown/ck08
Counting on ./Brown/ck09
Counting on ./Brown/ck10
Counting on ./Brown/ck11
Counting on ./Brown/ck12


In [9]:
# problem 4.2
print( "Running on problem 4.2" )
d = transitProb( filePath = "./4_1", encoding = "UTF-8", lambd = 0.1 )
for ( k, v ) in d.items():
    print( k, v )

Running on problem 4.2
<EOS>|. 0.9017836250564386
nn|at 0.49356062461227024
at|in 0.3587854174474563
in|nn 0.2770313419976999
nn|jj 0.4433386291775002
.|nn 0.1301956366945673
jj|at 0.19883497404251152
,|nn 0.11984927621373036
nn|in 0.14282314741419053
in|nns 0.2629583700464489
vb|to 0.8213011346171837
nns|jj 0.18793639992883385
nn|nn 0.07635915639789297
in|vbn 0.35757387098319043
cc|nn 0.059410270565850194
cc|, 0.1548036712631908
nn|pp$ 0.5104259016277173
at|<SOS> 0.1445805176746115
jj|in 0.06658214791500144
.|nns 0.13900401396730086
pp$|in 0.06345622547767744
np|np 0.2215048344021736
,|nns 0.13363755361927865
nns|at 0.07361866163842362
np|in 0.0548329911678181
vbd|pps 0.3374297829555638
at|vb 0.17913539851335492
nns|nn 0.03924207992006158
pps|<SOS> 0.1034216570187881
.|. 0.0974385931377234
at|cs 0.25403327565073186
,|np 0.16180619521831568
nns|in 0.04492456328293164
at|, 0.09229873168921404
in|rb 0.14499854839858237
vb|md 0.41103827413771
in|<SOS> 0.0835043929503689
,|rb 0.13070164387

nn$|pp$ 0.010408170502500088
bez|rb 0.004823151125401929
nns|jjt 0.16733181299885974
cd|bedz 0.017872003572370957
--|rb 0.004795762419408732
vbn|pp$ 0.010289963001052045
nns-tl|np-tl 0.04256836513869762
,|in 0.001435271018304473
hv|rb 0.004740985007422339
np|wdt 0.030627953601603895
cs|-- 0.049559726567025834
cs|`` 0.019145918688937915
ap|nn 0.0011152825841510543
vbn|vb 0.005041434007895579
np-hl|<SOS> 0.0029640652826230145
dts|cs 0.007665476963011031
beg|in 0.0014021047324973218
at|np 0.004898127067438317
rb|ben 0.06677524429967427
to|rp 0.027755762499174427
pp$|rb 0.004604041477456355
qlp|jj 0.0026234717223770744
at|: 0.10408620904447488
cc|) 0.07158248577831408
,|ppl 0.1297250859106529
--|jj 0.002592258495460036
jj-tl|, 0.0028366040471862466
nn-hl|at-hl 0.4351607801792304
nr|nn 0.001075942810459659
pn|, 0.0028022417934347476
ql|* 0.035072251849303286
in|dts 0.06570254592330002
vbg|* 0.035072251849303286
in|ql 0.018571233375842596
in|ben 0.06439183284340987
nn$-tl|at 0.00165397361804

np-tl|cc 0.001670841563971254
nns-hl|in-hl 0.11361181130716601
to|wdt 0.011295288557926393
pp$$|in 0.000523198158607812
bed|nps 0.04771627344222625
ppss+hv|`` 0.0071023366800234125
hvn|hvd 0.012767076723858855
''|vbd 0.002368927001953125
<EOS>|cd-hl 0.12637362637362637
bedz|'' 0.007027748857000589
od|in 0.0005149065871560242
hv|md* 0.06798773812130501
vbd|-- 0.017987486965589154
cd|bed 0.01865200937105785
np$|at 0.0006336320893329416
bez|) 0.02676262713325289
dti|vbg 0.003461461282914539
bez|<SOS> 0.001082119071433799
:-hl|nn-hl 0.04089831401475237
wps|<SOS> 0.001082119071433799
ppss+bem|<SOS> 0.0010646936435524173
beg|bez 0.0060414895089682995
np|jj-tl 0.014707298286154441
vbg|ppss+bem 0.1925015752993069
pps|ppo 0.005441558904207189
rbr|rb 0.001673449936184315
vbn|hvd* 0.4173497267759563
bez|'' 0.006914580598433751
abn|ppss 0.00441174346903115
cd-tl|np-tl 0.015025575447570333
do|<SOS> 0.0010646936435524173
cc|ppl 0.047719462667916275
jj|wql 0.27350044762757386
``|to 0.0040159300787149

nr|np 0.0009298041328490241
pp$|jj 0.0005009722920184658
vb|jj 0.0005009722920184658
hvz|dt 0.0035649238150237666
jj-tl|cd 0.002367710622980808
wrb|nn-tl 0.0023920592574928836
dti|rp 0.005300178323756687
cc|jj-tl 0.007726747544771809
jjr|nns 0.0005819708688226784
md|) 0.01383382175486985
.|bez 0.0031740067633041313
np|'' 0.0036327010999954736
.-hl|.-hl 0.0497365974589402
``|be 0.0050098323813091115
nns-tl|`` 0.003613074602674351
hvd|<SOS> 0.0005593562349923502
dti|ppo 0.0028588222720957574
ap|ppo 0.0028588222720957574
vbg|dti 0.010813906481606253
ql|ap 0.003354442284782745
pn|wdt 0.005746097665759703
bedz|<SOS> 0.0005593562349923502
,|pp$$ 0.15184484389782404
cd-hl|nn-hl 0.02114067439409905
vbz|`` 0.0035005177614695422
jj|nn-tl 0.0023175402775086817
vbn|) 0.013402861575590416
hvz|ex 0.014063489192366826
)|vbn 0.0010638516217750928
md|-- 0.009008226161510834
wrb|bedz 0.0031562709318610836
np|ber 0.0070260256641966395
do|wrb 0.006825564041787377
dts|vbd 0.001186370849609375
vbg|bem 0.113

to-hl|wrb-hl 0.2290167865707434
.|hv 0.004804547970015596
rb|uh 0.02914250839182179
vb+ppo|<SOS> 0.0003328256725343891
(|ppo 0.0016119838979729972
to|np-tl 0.0044511115482982495
nr|rp 0.0029885740704048615
cc-tl|jj-tl 0.00435682649720778
vb|cd 0.0013350642453567796
vbg|jjr 0.009025630796848509
wp$|cc 0.0004792746799980935
ppss|vbg 0.0010088961227174421
to|od 0.009130347054075868
ppss+md|vb 0.0005364488862017048
ap|nn-tl 0.0013487935377140558
ppss+md|rb 0.0004957355784768593
ql|ppss+ber 0.05562384757221882
nn$-hl|<SOS> 0.00031540024465300746
ql|wrb 0.003972434378017734
cs|do 0.01292487860611254
ap|rp 0.0029885740704048615
np$-tl|jj-tl 0.00435682649720778
fw-nn-tl|fw-jj-tl 0.14909390444810544
``|bed 0.005436414969664204
ber|'' 0.0020483454800597533
*|ppss+ber 0.05562384757221882
in|pps+bez 0.03791369920402179
fw-nn|fw-jj 0.1802788844621514
vb-tl|`` 0.002037278825807033
do*|nns 0.0003281517983081146
at|pn 0.006907342390474737
--|dt 0.0020101283816800677
ql|wdt 0.0032400114563941006
cc|np$

...

``|ex 0.0004974224473184408
nps|nns 1.994292696900144e-05
md-hl|`` 0.0001238125253252893
pps+hvz|vb 3.2601866012258306e-05
md*-hl|<SOS> 1.916797066951979e-05
vb-hl|md*-hl 0.022727272727272728
nns|vbg-hl 0.005687693898655637
dt$|pn 0.00041978323920012213
hvn|, 1.889923956332448e-05
fw-in|jj 1.7167274804371103e-05
nn|fw-dt 0.022267206477732792
jjr|nr 0.0006817900086773273
dti|nps 0.0008318209316394435
vbz|pp$ 6.501412579642304e-05
nns$-tl|in-tl 0.0007215953817895566
rb-hl|np-hl 0.001948972360028349
nn$-tl|np 3.1862446920060016e-05
ppss+ber|' 0.0030186608122941827
nps-hl|vbz-hl 0.009212730318257957
'|pps+bez 0.0023041474654377885
uh-tl|ap-tl 0.016819571865443424
,-hl|nns-tl-hl 0.017915309446254073
dt-hl|jj-hl 0.0024808299503834013
vb-hl|to 7.350287997647908e-05
cd|doz 0.0021384136858475897
nn-hl|be-hl 0.018211920529801324
qlp|<SOS> 1.916797066951979e-05
to-hl|ap-hl 0.012585812356979404
jjr-hl|vbz-hl 0.009212730318257957
nn-hl|rp-hl 0.017915309446254073
jjt-hl|nn-hl 0.0007244467860906218
(

rn|in-hl 0.0019805545552754773
<EOS>|rn 0.01950354609929078
wdt+bez-tl|`` 0.0001238125253252893
jj-tl|wdt+bez-tl 0.022727272727272728
nrs|ap 0.0001149497356156081
ber|nrs 0.017350157728706624
.-hl|np-tl 0.00027050954160928584
ex|np-tl 0.00027050954160928584
.-hl|jj-tl-hl 0.014986376021798364
cd-tl|nns 1.994292696900144e-05
nr|cd-tl 0.0011635286651152951
.-hl|nr-tl-hl 0.02099236641221374
cc-tl-hl|np-hl 0.001948972360028349
np-hl|cc-tl-hl 0.022267206477732792
np-hl|in-tl-hl 0.020599250936329586
at-tl-hl|<SOS> 1.916797066951979e-05
nn-tl-hl|at-tl-hl 0.02099236641221374
cc-tl-hl|nn-tl-hl 0.006235827664399093
nn-tl-hl|cc-tl-hl 0.022267206477732792
,-hl|jj-tl-hl 0.014986376021798364
,-hl|nps-hl 0.019855595667870037
nps-hl|,-hl 0.005036630036630037
:-hl|nn-tl-hl 0.006235827664399093
od|abn 0.00035978282200562574
np$-hl|np-hl 0.001948972360028349
rb-hl|jj-hl 0.0024808299503834013
np-hl|jjr-hl 0.017080745341614908
fw-in|np 3.1862446920060016e-05
hvn|ben 0.00043695876698180667
ber|vbz 0.00014823

wql|rbr 0.0008947454042622417
jj-tl-hl|: 0.0006851874922137785
dti|abx 0.0014149729868793415
hvz|do 0.0007854898600399886
jj-nc|vb-nc 0.01244343891402715
fw-at+nn-tl|fw-in 0.00837138508371385
,|fw-at+nn-tl 0.018211920529801324
ap-tl|np$-tl 0.005838641188959661
doz|vbg 6.131412900492743e-05
vbd|fw-jj-tl 0.009060955518945634
abx|* 0.0002365387923619474
hv|: 0.0006851874922137785
rb-nc|' 0.0030186608122941827
'|rb-nc 0.014986376021798364
vbz-tl|rb-tl 0.012585812356979404
cc|vbz-tl 0.017080745341614908
vbz|beg 0.0014998636487592038
ppls|rb 3.012757659251631e-05
pps|ppls 0.0028032619775739047
beg|pn$ 0.008064516129032258
pp$|dts 0.00044311956171446987
fw-nn$-tl|fw-nn-tl 0.005059797608095677
''|fw-nn$-tl 0.021400778210116732
fw-nn$-tl|fw-nn 0.0032796660703637453
fw-in|fw-nn$-tl 0.021400778210116732
nns-tl|nns 1.994292696900144e-05
do|nr-tl 0.00308641975308642
''|fw-vbg 0.02022058823529412
pps-tl|, 1.889923956332448e-05
doz*-tl|pps-tl 0.020599250936329586
vb|doz*-tl 0.022727272727272728
ppo-t

,|ap+ap-nc 0.022727272727272728
vb+jj-nc|, 1.889923956332448e-05
,|vb+jj-nc 0.022727272727272728
vb+vb-nc|, 1.889923956332448e-05
)|vb+vb-nc 0.022727272727272728
dti-tl|in-tl 0.0007215953817895566
np-tl|dti-tl 0.022267206477732792
(|pp$$ 0.005203405865657522
'|( 0.00047590205070520034
cd-tl|' 0.0030186608122941827
beg|nn-tl 8.197087798262218e-05
wpo|nps 0.0008318209316394435
np-hl|vbn-hl 0.005965292841648591
vbn-hl|in-tl 0.0007215953817895566
np-tl|vbn-hl 0.005965292841648591
hvz|wpo 0.0033598045204642642
at-hl|-- 0.0003186189317576179
)|fw-at+nn-tl 0.018211920529801324
rp|wrb 0.00024141866385743135
jj|fw-nn 0.0032796660703637453
jj-hl|at 1.1223756815881414e-05
vbz|nn-hl 0.0007244467860906218
jj|fw-dt 0.022267206477732792
ap-nc|`` 0.0001238125253252893
''|ap-nc 0.022267206477732792
hvz|: 0.0006851874922137785
nn-hl|nns$-hl 0.021400778210116732
jjs|`` 0.0001238125253252893
nn$-hl|at-hl 0.002899314707432789
,|fw-ppl 0.01950354609929078
*|'' 0.0001244850844235209
abx|pps 6.010797578195012

''|doz* 0.008064516129032258
ql|dod* 0.002447708055184691
jjr|ex+bez 0.007217847769028871
nr|* 0.0002365387923619474
vbd|pps+hvz 0.012168141592920354
ppss+md|hvd* 0.007513661202185793
in|nns$ 0.003613666228646518
ppss|bedz* 0.005461767626613705
pps+hvz|abn 0.00035978282200562574
wdt+bez|cs 4.957098565145288e-05
.|ppss+vb 0.022267206477732792
nns-tl|ppo 9.79658722525026e-05
,|dtx 0.00726552179656539
pps+bez|dt 0.00012216249833414778
vbg|dtx 0.00726552179656539
ex+hvz|, 1.889923956332448e-05
vb|pps+hvd 0.00843558282208589
od|abx 0.0014149729868793415
pp$|nn$ 0.000720178080398062
ppss|pps 6.010797578195012e-05
md+hv|vbd 4.1961669921875e-05
hvg|`` 0.0001238125253252893
abn|hvg 0.003349573690621194
wrb+ber|`` 0.0001238125253252893
ppss|wrb+ber 0.022727272727272728
ppo|vb+ppo 0.009290540540540541
rbt|rp 0.0001816260484776435
nns+md|pp$ 6.501412579642304e-05
abn|nns+md 0.022267206477732792
''|ber* 0.011652542372881356
rb+bez|`` 0.0001238125253252893
ppss+bez|cc 2.9127190497121706e-05
vbg|ppss

In [10]:
# problem 4.3
print( "Running on problem 4.3" )
d = emitProb( filePath = "./4_1", encoding = "UTF-8", lambd = 0.1 )
for ( k, v ) in d.items():
    print( k, v )

Running on problem 4.3
the/at 0.6355513517484572
,/, 0.9991357893181497
<SOS>/<SOS> 0.9991757772612107
<EOS>/<EOS> 0.9991757772612107
./. 0.8043466797615242
of/in 0.28907817625227605
and/cc 0.7328427608339908
a/at 0.22268035556861593
in/in 0.15733339745481922
to/to 0.980869204966122
to/in 0.09040383269598788
is/bez 0.9864239523800108
was/bedz 0.9913430896949277
``/`` 0.9946760614110126
''/'' 0.9946471413697887
for/in 0.0693432412084468
with/in 0.05798378831949747
The/at 0.0686189881477128
he/pps 0.3585222180935936
that/cs 0.2892737399956738
his/pp$ 0.3793337825218388
be/be 0.9877797546586761
on/in 0.04790123743412347
;/. 0.09165466487820795
as/cs 0.25020279039584686
I/ppss 0.3703481739281124
by/in 0.041632809416571874
at/in 0.04101923312913957
had/hvd 0.9841574943347363
?/. 0.07626381304234627
not/* 0.9494022019611218
are/ber 0.9753072474245438
from/in 0.03471763882578082
or/cc 0.1086205892165845
this/dt 0.44012927013460085
have/hv 0.9755244755244755
it/pps 0.208908002010885
which/wdt 

having/hvg 0.7828867235079173
week/nn 0.0016791526737277192
President/nn-tl 0.019084310773954127
name/nn 0.0016791526737277192
already/rb 0.006986858898864464
whole/jj 0.003981247093268243
political/jj 0.003965640479809724
experience/nn 0.0016660394158305872
shall/md 0.02028304910886011
turned/vbd 0.009654998779296875
seems/vbz 0.03397390976227697
light/nn 0.0016463695289848897
State/nn-tl 0.018637196894048913
whose/wp$ 0.8353373413493654
across/in 0.0020571388771885605
ago/rb 0.006740360544925694
enough/qlp 0.7979896238651103
white/jj 0.0038407875721415707
became/vbd 0.009349822998046875
today/nr 0.15191521011528447
let/vb 0.007264288508731372
far/rb 0.006712971838932497
known/vbn 0.008384245417912387
moment/nn 0.0016004731263449285
five/cd 0.017783645831796657
keep/vb 0.0071457362686867965
available/jj 0.0037471478913904555
tell/vb 0.007116098208675653
service/nn 0.001574246610550665
field/nn 0.001574246610550665
why/wrb 0.05247563866210166
problems/nns 0.004334867125716586
To/to 0.0

appeared/vbd 0.004505157470703124
respect/nn 0.0007743378788256291
medical/jj 0.001827534435992596
J./np 0.0033919023039445705
persons/nns 0.002123015225518244
nations/nns 0.002123015225518244
appear/vb 0.0034706168273049517
certainly/rb 0.003207217471803327
character/nn 0.0007677812498770632
trial/nn 0.0007612246209284973
rather/in 0.0009626514455525669
international/jj 0.001811927822534077
running/vbg 0.006471427615883703
plant/nn 0.0007612246209284973
Just/rb 0.00317982876581013
red/jj 0.001811927822534077
Well/uh 0.17714372902044553
generally/rb 0.003152440059816934
audience/nn 0.0007546679919799315
technical/jj 0.0017963212090755577
ask/vb 0.003411340707282664
kept/vbd 0.004390716552734374
changes/nns 0.002086755358301878
responsibility/nn 0.0007546679919799315
image/nn 0.0007546679919799315
throughout/in 0.0009543598741007791
man's/nn$ 0.07535681550346994
earth/nn 0.0007546679919799315
fiscal/jj 0.0017807145956170386
report/nn 0.0007481113630313656
volume/nn 0.0007481113630313656

agreement/nn 0.0005186293498315601
manager/nn 0.0005186293498315601
apartment/nn 0.0005186293498315601
Old/jj-tl 0.01904005391873676
practice/nn 0.0005186293498315601
London/np 0.0022911995921606793
bridge/nn 0.0005186293498315601
enter/vb 0.002314732486870339
bill/nn 0.0005120727208829943
outside/in 0.0006475717303846294
requirements/nns 0.001415947814799102
25/cd 0.0057606915780311855
houses/nns 0.001415947814799102
lead/vb 0.002314732486870339
trip/nn 0.0005120727208829943
while/nn 0.0005120727208829943
neither/cc 0.002068030525295641
sea/nn 0.0005120727208829943
someone/pn 0.029804609983208667
reality/nn 0.0005120727208829943
expression/nn 0.0005120727208829943
distribution/nn 0.0005055160919344284
citizens/nns 0.001397817881190919
session/nn 0.0005055160919344284
traditional/jj 0.0012032698976518288
teacher/nn 0.0005055160919344284
district/nn 0.0005055160919344284
attack/nn 0.0005055160919344284
essential/jj 0.0012032698976518288
inside/in 0.0006392801589328416
Its/pp$ 0.00455689

included/vbn 0.0018164154699761232
taste/nn 0.0003481569971688476
spend/vb 0.0015737809865917416
January/np 0.0015380872104138062
win/vb 0.0015737809865917416
liberal/jj 0.0008287111746473686
seat/nn 0.0003481569971688476
contract/nn 0.0003481569971688476
percent/nn 0.0003481569971688476
device/nn 0.0003481569971688476
No./nn-tl 0.003956957837161125
p.m./rb 0.0014543402882387418
contact/nn 0.0003481569971688476
hole/nn 0.0003481569971688476
straight/rb 0.0014543402882387418
save/vb 0.0015737809865917416
leg/nn 0.0003481569971688476
Once/rb 0.0014543402882387418
once/cs 0.0023929266709928616
wonderful/jj 0.0008287111746473686
finished/vbn 0.0018164154699761232
flowers/nns 0.0009626994745945241
dress/nn 0.0003481569971688476
generation/nn 0.0003481569971688476
properly/rb 0.0014543402882387418
approximately/rb 0.0014543402882387418
suggest/vb 0.0015737809865917416
objective/nn 0.0003481569971688476
create/vb 0.0015737809865917416
chest/nn 0.0003481569971688476
rise/nn 0.00034815699716884

...

aftuh/cs 4.957098565145288e-05
majuh/nn 7.2122918434224565e-06
tole/vbd 4.1961669921875e-05
thet/cs 4.957098565145288e-05
t'gethuh/rb 3.012757659251631e-05
f'ovuh/in+in 0.022727272727272728
yeah/nn 7.2122918434224565e-06
behahn/in 9.120728596966613e-06
doan/do* 0.002066115702479339
thet's/dt+bez 0.004858657243816255
nahce/jj 1.7167274804371103e-05
d'you/do+ppss 0.022727272727272728
Taos/np 3.1862446920060016e-05
eventshahleh/rb 3.012757659251631e-05
eventshah-leh/rb-nc 0.014986376021798364
re-echo/vb 3.2601866012258306e-05
Moriarty's/np$ 0.0004210687490430256
dust-swirling/jj 1.7167274804371103e-05
shards/nns 1.994292696900144e-05
unwired/vbd 4.1961669921875e-05
loosely-taped/jj 1.7167274804371103e-05
side-rack/nn 7.2122918434224565e-06
western-style/jj 1.7167274804371103e-05
flying-mount/nn 7.2122918434224565e-06
slivery/jj 1.7167274804371103e-05
truck's/nn$ 0.000720178080398062
glassless/jj 1.7167274804371103e-05
pearly/jj 1.7167274804371103e-05
roar/vb 3.2601866012258306e-05
anteate

parquet/nn 7.2122918434224565e-06
sheeted/vbn 3.7628192410051514e-05
daubed/vbd 4.1961669921875e-05
Ruffians/nns 1.994292696900144e-05
slave's/nn$ 0.000720178080398062
Dandy's/np$ 0.0004210687490430256
adjudged/vbn 3.7628192410051514e-05
soirees/nns 1.994292696900144e-05
hunts/nns 1.994292696900144e-05
abated/vbd 4.1961669921875e-05
Lalaurie's/np$ 0.0004210687490430256
Dominique/np 3.1862446920060016e-05
octoroon/nn 7.2122918434224565e-06
cameo-like/jj 1.7167274804371103e-05
slashes/nns 1.994292696900144e-05
sibilant/jj 1.7167274804371103e-05
hacksaw/nn 7.2122918434224565e-06
Vivaldi/np 3.1862446920060016e-05
clobbered/vbd 4.1961669921875e-05
Embarcadero/np 3.1862446920060016e-05
sucker-rolling/jj 1.7167274804371103e-05
freight-jumper/nn 7.2122918434224565e-06
lonelier/jjr 0.0005485189987035006
cannonball/nn 7.2122918434224565e-06
poshest/jjt 0.0010452299505891295
askin'/vbg 6.131412900492743e-05
conning/vbg 6.131412900492743e-05
freights/nns 1.994292696900144e-05
hooch/nn 7.2122918434

Drive/vb 3.2601866012258306e-05
double-married/jj 1.7167274804371103e-05
parades/vbz 0.00014823998706269207
mumbling/vbg 6.131412900492743e-05
Lie/vb 3.2601866012258306e-05
gnarled/jj 1.7167274804371103e-05
talons/nns 1.994292696900144e-05
delineaments/nns 1.994292696900144e-05
tracings/nns 1.994292696900144e-05
refracted/vbn 3.7628192410051514e-05
unnnt/uh 0.001678364357644187
Sssshoo/uh 0.001678364357644187
whip's/nn$ 0.000720178080398062
unbent/vbn 3.7628192410051514e-05
glorying/vbg 6.131412900492743e-05
mastiff/nn 7.2122918434224565e-06
ranted/vbd 4.1961669921875e-05
prophesied/vbd 4.1961669921875e-05
paraded/vbd 4.1961669921875e-05
grandly/rb 3.012757659251631e-05
attis/fw-nn 0.0032796660703637453
skeletons/nns 1.994292696900144e-05
paxam/fw-nn 0.0032796660703637453
arrowheads/nns 1.994292696900144e-05
Swan/nn-tl 8.197087798262218e-05
Necklace/nn-tl 8.197087798262218e-05
timidly/rb 3.012757659251631e-05
Wolf/nn-tl 8.197087798262218e-05
chemistries/nns 1.994292696900144e-05
fermen

low-priced/jj 1.7167274804371103e-05
bludgeon/vb 3.2601866012258306e-05
chromed/vbn 3.7628192410051514e-05
too-expensive/jj 1.7167274804371103e-05
Allstates-Zenith/np 3.1862446920060016e-05
Financing/vbg 6.131412900492743e-05
Heads/nns 1.994292696900144e-05
availing/vbg 6.131412900492743e-05
disquietude/nn 7.2122918434224565e-06
encircled/vbn 3.7628192410051514e-05
Spike-haired/jj 1.7167274804371103e-05
red-faced/jj 1.7167274804371103e-05
decked/vbn 3.7628192410051514e-05
horn-rimmed/jj 1.7167274804371103e-05
Hinkle/np 3.1862446920060016e-05
fancying/vbg 6.131412900492743e-05
Mormon/np-tl 0.00027050954160928584
Beard/nn-tl 8.197087798262218e-05
instigation/nn 7.2122918434224565e-06
secured/vbd 4.1961669921875e-05
Cal/np 3.1862446920060016e-05
Ye/at-tl 0.0013864381144441645
Olde/jj 1.7167274804371103e-05
Gasse/nn-tl 8.197087798262218e-05
Filling/vbg-tl 0.006097560975609756
Avocado/nn-tl 8.197087798262218e-05
Capistrano/np 3.1862446920060016e-05
By-the-Sea/np 3.1862446920060016e-05
Drive

Travancore/np 3.1862446920060016e-05
subcontinent/nn 7.2122918434224565e-06
Kali/np 3.1862446920060016e-05
worshiped/vbn 3.7628192410051514e-05
Thuggee/np 3.1862446920060016e-05
Nuf/ap 0.0001149497356156081
sed/vbn 3.7628192410051514e-05
amulet/nn 7.2122918434224565e-06
housebreakers/nns 1.994292696900144e-05
mem/fw-nn 0.0032796660703637453
rajah/nn 7.2122918434224565e-06
Inscribed/vbn 3.7628192410051514e-05
Balinese/np 3.1862446920060016e-05
Tjokorda/np 3.1862446920060016e-05
Agoeng/np 3.1862446920060016e-05
Whosoever/wps 0.0002769804099310067
rooftree/nn 7.2122918434224565e-06
cocu/fw-nn 0.0032796660703637453
fishmongers/nns 1.994292696900144e-05
vandals/nns 1.994292696900144e-05
belonging/nn 7.2122918434224565e-06
dogs/vbz 0.00014823998706269207
forty-fifth/od 0.0005548829701372075
Sort/nn 7.2122918434224565e-06
preceding/nn 7.2122918434224565e-06
pap/nn 7.2122918434224565e-06
gorging/vbg 6.131412900492743e-05
pinball/nn 7.2122918434224565e-06
Ubermenschen/fw-nns 0.00843558282208589

In [11]:
# problem 4.4
print( "Running on problem 4.4" )
problem4_4( filePath = "./4_1", encoding = "UTF-8" )

Running on problem 4.4
("recurring the active foreign fibers the endurance or There should figure When I insisted That banner from new , what they was The decade except warning in spine-chilling key is as could be impossible parents of her group though all that can there ''", 2.487066088315548e-125)
('a trees , terrifying of this .', 7.230991871944728e-17)
('flat War is not the human glass Is longer needs of bed ;', 1.5171435697638613e-38)
('it was taking into the all it is how There old mad world helps our substantive cm , would tell Republic 3 in the anti-secrecy At Another of various worried ones to get insuperably Great law .', 2.153971593301747e-113)
("he was included that more of the pawnshop to look with 1928 other on early feet They will they Don't set out in following with the character both the entire dome . .", 5.742447116607029e-83)


In [12]:
# problem 4.5
print( "Running on problem 4.5" )
problem4_5( filePath = "./4_1", encoding = "UTF-8", savePath = "./4_5",
            testFilePath = "." )

Running on problem 4.5
Ori: ['We', 'have', 'learned', 'much', 'about', 'interstellar', 'drives', 'since', 'a', 'hundred', 'years', 'ago', ';', ';', 'that', 'is', 'all', 'I', 'can', 'tell', 'you', 'about', 'them', '.', '<EOS>']
Tag: ['ppss', 'hv', 'vbn', 'ql', 'rb', 'jj', 'nns', 'in', 'at', 'cd', 'nns', 'rb', '.', '.', 'wps', 'bez', 'abn', 'ppss', 'md', 'vb', 'ppo', 'in', 'ppo', '.', '<EOS>']
Ori: ['Macneff', 'stopped', 'pacing', 'to', 'stare', 'at', 'Hal', 'with', 'his', 'pale', 'blue', 'eyes', '.', '<EOS>']
Tag: ['np', 'vbd', 'vbg', 'to', 'vb', 'in', 'np', 'in', 'pp$', 'jj', 'jj', 'nns', '.', '<EOS>']
Ori: ['``', 'How', 'did', 'you', 'know', "''", '?', '?', 'He', 'said', 'sharply', '.', '<EOS>']
Tag: ['``', 'wrb', 'dod', 'ppss', 'vb', "''", '.', '.', 'pps', 'vbd', 'rb', '.', '<EOS>']
Ori: ['``', 'Forgive', 'me', ',', 'Sandalphon', "''", ',', 'said', 'Hal', '.', '<EOS>']
Tag: ['``', 'vb', 'ppo', ',', 'np', "''", ',', 'vbd', 'np', '.', '<EOS>']
Ori: ['``', 'But', 'it', 'was', 'inevitabl

Tag: ['rb', ',', 'pps', 'vbd', ',', '``', 'rb', ',', 'rb', 'cd', 'at-nc', 'nn-nc', 'vbz', ',', 'cc', 'pps', 'bez', 'ql', 'jj', 'in', 'dt', 'nn', '.', '<EOS>']
Ori: ['``', 'There', 'will', 'be', 'no', 'women', 'aboard', 'the', 'Gabriel', '.', '<EOS>']
Tag: ['``', 'ex', 'md', 'be', 'at', 'nns', 'in', 'at', 'np', '.', '<EOS>']
Ori: ['And', ',', 'if', 'a', 'man', 'is', 'married', ',', 'he', 'will', 'automatically', 'be', 'given', 'a', 'divorce', "''", '.', '<EOS>']
Tag: ['cc', ',', 'cs', 'at', 'nn', 'bez', 'vbn', ',', 'pps', 'md', 'rb', 'be', 'vbn', 'at', 'nn', "''", '.', '<EOS>']
Ori: ["It's", 'inevitable', 'in', 'this', 'case', ',', 'for', 'the', 'couple', 'will', 'be', 'separated', 'for', ',', 'at', 'the', 'least', ',', 'forty', 'years', '.', '<EOS>']
Tag: ['pps+bez', 'jj', 'in', 'dt', 'nn', ',', 'in', 'at', 'nn', 'md', 'be', 'vbn', 'in', ',', 'in', 'at', 'ap', ',', 'cd', 'nns', '.', '<EOS>']
Ori: ['Naturally', ',', 'he', 'couched', 'the', 'provision', 'in', 'obscure', 'language', '.', 

Tag: ['rb', ',', 'in', 'abn', 'jj', 'nns', ',', 'pps', 'bedz', 'at', 'jj', 'nn', 'cc', 'in', 'rb', 'hvd', 'pp$', 'jj', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', '.', '<EOS>']
Ori: ['Its', 'citizens', 'spoke', 'all', 'of', 'the', "world's", 'surviving', 'tongues', ',', 'plus', 'a', 'new', 'one', 'called', 'Lingo', ',', 'a', 'pidgin', 'whose', 'vocabulary', 'was', 'derived', 'from', 'the', 'other', 'six', 'and', 'whose', 'syntax', 'was', 'so', 'simple', 'it', 'could', 'be', 'contained', 'on', 'half', 'a', 'sheet', 'of', 'paper', '.', '<EOS>']
Tag: ['pp$', 'nns', 'vbd', 'abn', 'in', 'at', 'nn$', 'vbg', 'nns', ',', 'cc', 'at', 'jj', 'cd', 'vbd', 'rp', ',', 'at', 'nn', 'wp$', 'nn', 'bedz', 'vbn', 'in', 'at', 'ap', 'cd', 'cc', 'wp$', 'nn', 'bedz', 'ql', 'jj', 'pps', 'md', 'be', 'vbn', 'in', 'abn', 'at', 'nn', 'in', 'nn', '.', '<EOS>']
Ori: ['Hal', 'saw', 'in', 'his', 'mind', 'the', 'rest', 'of', 'Earth', ':', 'Iceland', ',', 'Greenland', ',', 'the', 'Caribbean', 'Islands

Tag: ['np', 'bedz', 'ql', 'vbn', 'in', 'at', 'nn', 'cs', 'pps', '.', '<EOS>']
Ori: ['No', 'matter', 'who', 'the', 'candidate', 'for', 'marriage', 'selected', 'by', 'the', 'gapt', 'The', 'gapt', '.', '<EOS>']
Tag: ['at', 'nn', 'wps', 'at', 'nn', 'in', 'nn', 'vbn', 'in', 'at', 'nn', 'at', 'nn', '.', '<EOS>']
Ori: ['Pornsen', '.', '<EOS>']
Tag: ['np', '.', '<EOS>']
Ori: ['He', 'would', 'no', 'longer', 'have', 'to', 'see', 'that', 'fat', 'face', ',', 'hear', 'that', 'whining', 'voice', '``', 'Hal', 'Yarrow', "''", '!', '!', 'Said', 'the', 'whining', 'voice', '.', '<EOS>']
Tag: ['pps', 'md', 'ql', 'rbr', 'hv', 'to', 'vb', 'cs', 'jj', 'nn', ',', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', 'nil', '.', '<EOS>']
Ori: ['And', ',', 'slowly', ',', 'feeling', 'himself', 'icy', 'yet', 'burning', ',', 'Hal', 'turned', '.', '<EOS>']
Tag: ['cc', ',', 'rb', ',', 'vbg', 'ppl', 'bedz*', 'rb', 'vbg', ',', 'np', 'vbd', '.', '<EOS>']
Ori: ['There', 'was', 'the',

Tag: ['at', 'nn', 'wps', 'hvd', 'vbn', 'np', 'hvd', 'vbn', 'in', 'vbg', 'cd', 'cd', 'nns$', 'nns', 'in', 'at', 'jj', 'nn', 'in', 'jj', 'nns', '.', '<EOS>']
Ori: ['The', 'description', 'of', 'the', 'Siddo', 'syntax', 'was', 'very', 'restricted', '.', '<EOS>']
Tag: ['at', 'nn', 'in', 'at', 'od', 'nn', 'bedz', 'ql', 'vbn', '.', '<EOS>']
Ori: ['And', ',', 'as', 'Hal', 'found', 'out', ',', 'obviously', 'mistaken', 'in', 'many', 'cases', '.', '<EOS>']
Tag: ['cc', ',', 'cs', 'np', 'vbd', 'rp', ',', 'rb', 'vbn', 'in', 'ap', 'nns', '.', '<EOS>']
Ori: ['This', 'discovery', 'caused', 'Hal', 'anxiety', '.', '<EOS>']
Tag: ['dt', 'nn', 'vbd', 'np', 'nn', '.', '<EOS>']
Ori: ['His', 'duty', 'was', 'to', 'write', 'a', 'school', 'text', 'and', 'to', 'teach', 'the', 'entire', 'personnel', 'of', 'the', 'Gabriel', 'how', 'to', 'speak', 'Ozagen', '.', '<EOS>']
Tag: ['pp$', 'nn', 'bedz', 'to', 'vb', 'at', 'nn', 'nn', 'cc', 'to', 'vb', 'at', 'jj', 'nns', 'in', 'at', 'np', 'wrb', 'to', 'vb', 'ppo', '.', '<EOS>